In [2]:
import pandas as pd
from utils import persistence as ps
from urllib3.response import HTTPResponse
from typing import List, Dict, Tuple
import numpy as np
import pickle
from scipy.io import mmwrite, mmread

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from toolz import partition_all

In [5]:
#INPUT_BUCKET = 'dq-data'
data_folder = '/media/siri/78C6823EC681FD1E/minio/data/dq-data/'
#HASH_BUCKET = 'dq-hashed'

In [6]:
#load train_set
data = 'train.csv'
#filestream = ps.get_file_stream(bucket=INPUT_BUCKET, filename=data)
filestream = data_folder+data
dtypes = {
    'id': 'int64',
    'qid1': 'int64',
    'qid2': 'int64',
    'question1': 'object',
    'question2': 'object',
    'is_duplicate': 'int64'
}
df = pd.read_csv(#urlpath=s3_in_url, 
                                     #storage_options=s3_options,
                                     filestream,
                                     header=0, 
                                     usecols=dtypes.keys(), 
                                     #names=dtypes.keys(),
                                     skipinitialspace=True,
                                     skip_blank_lines=True,
                                     encoding='utf-8')
df = df.set_index('id')
df = df.dropna()

In [7]:
#del df

### Train-test split

In [ ]:
from sklearn.model_selection import train_test_split
#shrink df to 150,000 records
#df = df.iloc[:100000]

X = df.drop(columns=['is_duplicate'])

y = df['is_duplicate']
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train.info()

In [ ]:
import pickle
pickle.dump(X_train, open(data_folder+'X_train.p', 'wb'))
pickle.dump(y_train, open(data_folder+'y_train.p', 'wb'))
pickle.dump(X_test, open(data_folder+'X_test.p', 'wb'))
pickle.dump(y_test, open(data_folder+'y_test.p', 'wb'))

In [8]:
X_train = pickle.load(open(data_folder+'X_train.p', 'rb'))
X_test = pickle.load(open(data_folder+'X_test.p', 'rb'))

In [6]:
y_train = pickle.load(open(data_folder+'y_train.p', 'rb'))
y_test = pickle.load(open(data_folder+'y_test.p', 'rb'))

In [ ]:
del X,y,df

In [52]:
del X_train,X_test

In [7]:
del y_train, y_test

### Memory Check

In [8]:
import sys
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) 
        for x in dir() if not x.startswith('_') 
        and x not in sys.modules and x not in ipython_vars], 
       key=lambda x: x[1], reverse=True)

[('HTTPResponse', 1056),
 ('Dict', 888),
 ('List', 888),
 ('Tuple', 888),
 ('dtypes', 368),
 ('assign_pwmetric', 136),
 ('cdist', 136),
 ('compute_pairwise_dist', 136),
 ('compute_pairwise_kernel', 136),
 ('compute_pairwise_metric', 136),
 ('directed_hausdorff', 136),
 ('laplacian_kernel', 136),
 ('linear_kernel', 136),
 ('mmread', 136),
 ('mmwrite', 136),
 ('pairwise_distances', 136),
 ('partition_all', 136),
 ('polynomial_kernel', 136),
 ('procrustes', 136),
 ('rbf_kernel', 136),
 ('sigmoid_kernel', 136),
 ('filestream', 106),
 ('data_folder', 97),
 ('np', 80),
 ('pd', 80),
 ('ps', 80),
 ('data', 58)]

In [78]:
#del y_train, y_test

# Feature Extraction

### Tokenizing and preprocessing

In [10]:
from gensim.parsing.preprocessing import preprocess_string
def get_tokens(process='train'):
    if process=='test':
        X = X_test
    else:
        X = X_train
    series = pd.Series(pd.concat([X['question1'], X['question2']]),dtype=str)
    series.dropna()
    for question in series:
        yield preprocess_string(question)

### Word2Vec (fasttext)

In [11]:
from gensim.models import FastText
model = FastText.load_fasttext_format(data_folder+'cc.en.300.bin')

In [12]:
def get_ft_vectors(model, process):
    for tokens in get_tokens(process):
        vectors = []
        for token in tokens:
            try:
                vector = model.wv[token]
            except:
                continue
            vectors.append(vector)
        yield np.array(vectors)

In [13]:
def get_tfidf_for_valid_vectors(model, process):
    for tokens in get_tokens(process):
        tf_idf_tokens = []
        for token in tokens:
            try:
                vector = model.wv[token]
                tf_idf_tokens.append(token)
            except:
                continue
        yield np.array(tf_idf_tokens)

In [ ]:
#X_ft = np.array([vectors for vectors in get_ft_vectors(model,'train')])
#X_ft.shape

In [ ]:
# split back into two
#X1_ft = X_ft[:len(X_train)]
#X2_ft = X_ft[len(X_train):]

##### Test set

In [10]:
#X_ft_test = np.array([vectors for vectors in get_ft_vectors(model,'test')])

In [80]:
#X_ft_test.shape

In [12]:
#del model

In [13]:
# split back into two
#X1_ft_test = X_ft_test[:len(X_test)]
#X2_ft_test = X_ft_test[len(X_test):]

### TFIDF and Word2Vec

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
pass_through = lambda x:x
tfidf = TfidfVectorizer(analyzer=pass_through)
X_trfmd = tfidf.fit_transform(get_tfidf_for_valid_vectors(model,'train'))

In [15]:
X_trfmd

<541744x50696 sparse matrix of type '<class 'numpy.float64'>'
	with 2522482 stored elements in Compressed Sparse Row format>

In [16]:
# split back into two
X1_trfmd = X_trfmd[:len(X_train)]
X2_trfmd = X_trfmd[len(X_train):]

In [19]:
def get_weights_and_w2vectors(tfidf_matrix, tfidf_vectorizer, w2v_model):
    weights = []
    w2v = []
    rows = tfidf_matrix.shape[0]
    inverse_vocab_dict = {v: k for k, v in tfidf_vectorizer.vocabulary_.items()}
    for doc in range(rows):
        features = tfidf_matrix[doc,:].nonzero()[1]
        weights.append(np.array([tfidf_matrix[doc, x] for x in features]))
        w2v.append(np.array([w2v_model.wv[inverse_vocab_dict[x]] for x in features]))
    return np.array(weights), np.array(w2v)

In [20]:
X1_w, X1 = get_weights_and_w2vectors(X1_trfmd, tfidf, model)
X1_w.shape

(270872,)

In [21]:
X1.shape

(270872,)

In [22]:
X1_w.shape

(270872,)

In [23]:
X2_w, X2 = get_weights_and_w2vectors(X2_trfmd, tfidf, model)
X2_w.shape

(270872,)

In [24]:
pickle.dump(X1_w, open(data_folder+'X1_w.p','wb'))
pickle.dump(X2_w, open(data_folder+'X2_w.p','wb'))

In [25]:
pickle.dump(X1, open(data_folder+'X1.p','wb'))
pickle.dump(X2, open(data_folder+'X2.p','wb'))

In [1]:
X1_w[420].shape

NameError: name 'X1_w' is not defined

In [33]:
X1[420].shape

(8, 300)

In [23]:
del X1_w, X2_w, X1, X2

In [ ]:
# dimension reduction using SVD
#from sklearn.decomposition import TruncatedSVD
#import time
#start = time.time()
#svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
#X_svd = svd.fit_transform(X_trfmd)
#end =  time.time()
#print('created SVD transform in time {}'.format(end-start))

In [ ]:
#X_svd.shape

In [ ]:
# split back into two
#X1 = X_svd[:len(X_train), :]
#X2 = X_svd[len(X_train):, :]

##### Test set

In [34]:
X_test_trfmd = tfidf.transform(get_tfidf_for_valid_vectors(model, 'test'))

In [35]:
X_test_trfmd

<266830x50696 sparse matrix of type '<class 'numpy.float64'>'
	with 1229426 stored elements in Compressed Sparse Row format>

In [36]:
# split back into two
X1_test_trfmd = X_test_trfmd[:len(X_test)]
X2_test_trfmd = X_test_trfmd[len(X_test):]

In [37]:
X1_test_w, X1_test = get_weights_and_w2vectors(X1_test_trfmd, tfidf, model)
X1_test_w.shape

(133415,)

In [38]:
X2_test_w, X2_test = get_weights_and_w2vectors(X2_test_trfmd, tfidf, model)
X2_test_w.shape

(133415,)

In [39]:
pickle.dump(X1_test_w, open(data_folder+'X1_test_w.p','wb'))
pickle.dump(X2_test_w, open(data_folder+'X2_test_w.p','wb'))

In [40]:
pickle.dump(X1_test, open(data_folder+'X1_test.p','wb'))
pickle.dump(X2_test, open(data_folder+'X2_test.p','wb'))

In [54]:
del X1_test_w, X2_test_w, X1_test, X2_test

In [ ]:
# dimension reduction using SVD
#start = time.time()
#X_test_svd = svd.transform(X_test_trfmd)
#end =  time.time()
#print('created SVD transform in time {}'.format(end-start))

In [ ]:
# split back into two
#X1_test = X_test_svd[:len(X_test), :]
#X2_test = X_test_svd[len(X_test):, :]

In [ ]:
# build complete feature dataframe
#X_test_temp = pd.concat([pd.DataFrame(X1_test, columns=['q1_'+str(i) for i in range(X1_test.shape[1])], index=X_test.index), 
#                    pd.DataFrame(X2_test, columns=['q2_'+str(i) for i in range(X2_test.shape[1])], index=X_test.index)], axis=1)
#X_test_temp.head()

### Pairwise Metrics

In [14]:
#def get_q_lengths(X):
#    #q_meta = []
#    for q in X:
#        #q_meta.append(len(q))
#        yield len(q)
#    #return q_meta

In [ ]:
#q_meta_train = [(q1_len, q2_len) for q1_len, q2_len in zip(get_q_lengths(X1_ft), get_q_lengths(X2_ft))]

In [ ]:
#pickle.dump(q_meta_train, open(data_folder+'q_meta_train.p', 'wb'))

In [ ]:
#ps.copy_file(dest_bucket=INPUT_BUCKET, file='q_meta_train.p', source='q_meta_train.p')

In [ ]:
#X_train_300 = np.concatenate( 
#    np.vstack( [np.array(np.vsplit(y, y.shape[0])) for y in (x for x in X_ft if x.size>0)] )
#)

In [ ]:
#X_train_300.shape

In [ ]:
#mmwrite( data_folder+'wor2vec_300_full_train.mtx', X_train_300 )
#np.savez(data_folder+'wor2vec_300_full_train', data=X_train_300)

In [ ]:
#ps.copy_file(dest_bucket=INPUT_BUCKET, file='wor2vec_300_full_train.mtx', source='wor2vec_300_full_train.mtx')

In [ ]:
#del X_ft

In [ ]:
#ps.get_file(bucket=INPUT_BUCKET, filename='embed2_full_train.mtx', filepath='embed2_full_train.mtx')

In [ ]:
#X_rd = mmread('embed2_full_train.mtx')

In [ ]:
#X_rd.shape

In [ ]:
#ps.get_file(bucket=INPUT_BUCKET, filename='q_meta_train.p', filepath='q_meta_train.p')

In [47]:
from sklearn.metrics.pairwise import linear_kernel, polynomial_kernel, sigmoid_kernel, laplacian_kernel, rbf_kernel
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist, directed_hausdorff
from fastdtw import fastdtw
import similaritymeasures
from scipy.spatial import procrustes
def compute_pairwise_kernel(pc1, pc2, method='linear'):
    if method=='polynomial':
        return polynomial_kernel(pc1, pc2, 2)
    elif method=='rbf':
        return rbf_kernel(pc1, pc2)
    elif method=='sigmoid':
        return sigmoid_kernel(pc1, pc2)
    elif method=='laplacian':
        return laplacian_kernel(pc1, pc2)
    else:
        return linear_kernel(pc1, pc2)
    
def compute_pairwise_dist(pc1, pc2, w1, w2, method='euclidean'):
    if pc1.size == 0 or pc2.size == 0:
        return np.nan
    if method=='hausdorff':
        dist = directed_hausdorff(pc1, pc2)
        return dist[0]
    else:
        dist_mat = pairwise_distances(pc1, pc2, metric=method) 
    #dist_mat = cdist(pc1, pc2, metric=method)
    #return np.linalg.norm(dist_mat, ord='fro')
    return np.average(dist_mat, weights=np.matmul(w1.reshape(-1,1),w2.reshape(-1,1).T))

def compute_pairwise_metric(pc1, pc2, method='dtw'):
    if pc1.size == 0 or pc2.size == 0:
        return np.nan
    #if method=='fdtw':
    #    dist, _ = fastdtw(pc1, pc2, dist=euclidean)
    if method=='pcm':
        dist = similaritymeasures.pcm(pc1[:,:2], pc2[:,:2])
    if method=='discrete_frechet':
        dist = similaritymeasures.frechet_dist(pc1[:,:2], pc2[:,:2])
    if method=='area':
        dist = similaritymeasures.area_between_two_curves(pc1[:,:2], pc2[:,:2])
    if method=='curve_length':
        dist = similaritymeasures.curve_length_measure(pc1[:,:2], pc2[:,:2])
    if method=='dtw':
        dist, _ = similaritymeasures.dtw(pc1[:,:2], pc2[:,:2])
    #if method=='procrustes': 
    #    mbox1 = MinimumBoundingBox([x[:2] for x in pc1.tolist()])
    #    mbox2 = MinimumBoundingBox([x[:2] for x in pc2.tolist()])
    #    _,_,dist = procrustes(make_array(mbox1.corner_points), make_array(mbox2.corner_points))
    return dist

        
def assign_pwmetric(df, method='euclidean'):
    #return compute_pairwise_kernel(pc1_embd, pc2_embd, method=method)
    return df.apply(compute_pairwise_dist, method, axis=1)

In [46]:
def compute_delayed(X1, X2, X1_w, X2_w, method):
    temp = []
    for q_tuple in zip(X1, X2, X1_w, X2_w):
        if q_tuple:
            q1_rd, q2_rd, q1_w, q2_w = q_tuple
            temp.append(delayed(compute_pairwise_dist)(q1_rd, q2_rd, q1_w, q2_w, method))
        else:
            temp.append(delayed(np.nan))
    return compute(*temp)

In [10]:
def pickle_and_del(obj, file, data_folder=data_folder):
    pickle.dump(obj, open(data_folder+file+'.p', 'wb'))
    del obj

In [11]:
def compute_and_save(X1, X2, X1_w, X2_w, method, file, data_folder=data_folder):
    computed_obj = compute_delayed(X1, X2, X1_w, X2_w, method)
    pickle_and_del(computed_obj, file, data_folder)    

In [45]:
#q_meta_train = pickle.load(open(data_folder+'q_meta_train.p','rb'))

In [46]:
#len(q_meta_train)

In [47]:
#mmwrite( data_folder+'wor2vec_300_full_train.mtx', X_train_300 )
#X_train_300 = np.load(data_folder+'wor2vec_300_full_train.npz')['data']

In [48]:
#X_train_300.shape

In [49]:
# rebuild X1_rd and X2_rd
#X1_list = []
#X2_list = []
#X1_rd_list = []
#X2_rd_list = []
#q1_ptr = 0
#for len_q1, _ in q_meta_train:
#    q1 = np.array(X_train_300[q1_ptr:q1_ptr+len_q1])
#    #q1_rd = np.array(X_rd[q1_ptr:q1_ptr+len_q1])
#    X1_list.append(q1)
#    #X1_rd_list.append(q1_rd)
#    q1_ptr = q1_ptr+len_q1
#q2_ptr = q1_ptr
#for _, len_q2 in q_meta_train:
#    q2 = np.array(X_train_300[q2_ptr:q2_ptr+len_q2])
#    #q2_rd = np.array(X_rd[q2_ptr:q2_ptr+len_q2])
#    X2_list.append(q2)
#    #X2_rd_list.append(q2_rd)
#    q2_ptr = q2_ptr+len_q2
#X1 = np.array(X1_list)
#X2 = np.array(X2_list)
#X1_rd = np.array(X1_rd_list)
#X2_rd = np.array(X2_rd_list)

In [38]:
#del X1_list, X2_list, X_train_300, q_meta_train

#### Initialize Dask

In [12]:
import dask.dataframe as dd
from dask import delayed, compute
from dask.distributed import Client
from utils import dask
client = dask.create_dask_client(num_workers=8)

#### Compute Features

In [13]:
X1_w = pickle.load(open(data_folder+'X1_w.p','rb'))
X2_w = pickle.load(open(data_folder+'X2_w.p','rb'))

In [14]:
X1 = pickle.load(open(data_folder+'X1.p','rb'))
X2 = pickle.load(open(data_folder+'X2.p','rb'))

In [66]:
#jaccard = compute(*jaccard)

In [19]:
X2[171841].shape

(4, 300)

In [20]:
X2_w[171841].shape

(4,)

In [21]:
def compute_nd(X1, X2, X1_w, X2_w, method):
    temp = []
    for i,q_tuple in enumerate(zip(X1, X2, X1_w, X2_w)):
        if q_tuple:
            q1_rd, q2_rd, q1_w, q2_w = q_tuple
            temp.append(compute_pairwise_dist(q1_rd, q2_rd, q1_w, q2_w, i, method))
        else:
            temp.append(np.nan)
    return temp

In [37]:
X2_w[0].reshape(-1,1).T

array([[0.42336797, 0.32999582, 0.37884538, 0.54461761, 0.52127742]])

In [38]:
X1_w[0].reshape(-1,1) @ X2_w[0].reshape(-1,1).T

array([[0.08349811, 0.06508293, 0.07471721, 0.10741139, 0.10280816],
       [0.26338747, 0.2052984 , 0.23568889, 0.33881981, 0.32429931],
       [0.16017594, 0.12484976, 0.14333137, 0.20604921, 0.19721874],
       [0.11042219, 0.086069  , 0.09880987, 0.14204633, 0.13595878],
       [0.15579876, 0.12143795, 0.13941451, 0.20041844, 0.19182928],
       [0.11616445, 0.09054483, 0.10394826, 0.14943314, 0.14302902],
       [0.12695402, 0.09895481, 0.11360317, 0.16331277, 0.15631382],
       [0.10564753, 0.08234738, 0.09453733, 0.13590424, 0.13007992]])

In [40]:
temp = compute_nd(X1[:1], X2[:1], X1_w[:1], X2_w[:1], 'chebyshev')

(8, 5)
(8, 5)


In [ ]:
#chebyshev = compute(*chebyshev)
#compute_and_save(X1, X2, 'chebyshev', 'chebyshev_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'chebyshev', 'chebyshev_train_w', data_folder)

In [ ]:
#braycurtis = compute(*braycurtis)
#compute_and_save(X1, X2, 'braycurtis', 'braycurtis_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'braycurtis', 'braycurtis_train_w', data_folder)

In [ ]:
#cosine = compute(*cosine)
#compute_and_save(X1, X2, 'cosine', 'cosine_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'cosine', 'cosine_train_w', data_folder)

In [ ]:
#correlation = compute(*correlation)
#compute_and_save(X1, X2, 'correlation', 'correlation_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'correlation', 'correlation_train_w', data_folder)

In [ ]:
#hamming = compute(*hamming)

In [ ]:
#canberra = compute(*canberra)
#compute_and_save(X1, X2, 'canberra', 'canberra_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'canberra', 'canberra_train_w', data_folder)

In [ ]:
#hausdorff = compute(*hausdorff)
#compute_and_save(X1, X2, 'hausdorff', 'hausdorff_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'hausdorff', 'hausdorff_train_w', data_folder)

In [ ]:
#cityblock = compute(*cityblock)
#compute_and_save(X1, X2, 'cityblock', 'cityblock_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'cityblock', 'cityblock_train_w', data_folder)

In [ ]:
#euclidean = compute(*euclidean)
#compute_and_save(X1, X2, 'euclidean', 'euclidean_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'euclidean', 'euclidean_train_w', data_folder)

In [ ]:
#l1 = compute(*l1)
#compute_and_save(X1, X2, 'l1', 'l1_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'l1', 'l1_train_w', data_folder)

In [ ]:
#l2 = compute(*l2)
#compute_and_save(X1, X2, 'l2', 'l2_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'l2', 'l2_train_w', data_folder)

In [ ]:
#manhattan = compute(*manhattan)
#compute_and_save(X1, X2, 'manhattan', 'manhattan_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'manhattan', 'manhattan_train_w', data_folder)

In [ ]:
#dice = compute(*dice)

In [ ]:
#kulsinski = compute(*kulsinski)

In [ ]:
#rogerstanimoto = compute(*rogerstanimoto)

In [ ]:
#russellrao = compute(*russellrao)

In [ ]:
#sokalmichener = compute(*sokalmichener)

In [ ]:
#minkowski = compute(*minkowski)
#compute_and_save(X1, X2, 'minkowski', 'minkowski_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'minkowski', 'minkowski_train_w', data_folder)

In [ ]:
#seuclidean = compute(*seuclidean)
#compute_and_save(X1, X2, 'seuclidean', 'seuclidean_train', data_folder)

In [ ]:
#sokalsneath = compute(*sokalsneath)

In [38]:
#sqeuclidean = compute(*sqeuclidean)
#compute_and_save(X1, X2, 'sqeuclidean', 'sqeuclidean_train', data_folder)
compute_and_save(X1, X2, X1_w, X2_w, 'sqeuclidean', 'sqeuclidean_train_w', data_folder)

In [ ]:
#fdtw = compute(*fdtw)

In [ ]:
#dtw = compute(*dtw)

In [ ]:
#pcm = compute(*pcm)

In [ ]:
#area = compute(*area)

In [ ]:
#curve_length = compute(*curve_length)

In [ ]:
#discrete_frechet = compute(*discrete_frechet)

In [ ]:
#procrustes = compute(*procrustes)

In [39]:
del X1, X2, X1_w, X2_w

#### add above metrics to X_train

In [67]:
def add_column(df, column, train_or_test, data_folder=data_folder):
    col_arr = pickle.load(open(data_folder+column+'_'+train_or_test+'_w.p', 'rb'))
    return pd.concat([df,
                     pd.Series(col_arr, name=column,index=df.index)
                      ], axis=1)

In [68]:
X_train = add_column(X_train, 'chebyshev', 'train')

In [69]:
X_train = add_column(X_train, 'braycurtis', 'train')

In [70]:
X_train = add_column(X_train, 'cosine', 'train')

In [71]:
X_train = add_column(X_train, 'correlation', 'train')

In [72]:
X_train = add_column(X_train, 'canberra', 'train')

In [73]:
X_train = add_column(X_train, 'hausdorff', 'train')

In [74]:
X_train = add_column(X_train, 'cityblock', 'train')

In [75]:
X_train = add_column(X_train, 'euclidean', 'train')

In [76]:
X_train = add_column(X_train, 'l1', 'train')

In [77]:
X_train = add_column(X_train, 'l2', 'train')

In [78]:
X_train = add_column(X_train, 'manhattan', 'train')

In [79]:
X_train = add_column(X_train, 'minkowski', 'train')

In [80]:
X_train = add_column(X_train, 'sqeuclidean', 'train')

In [81]:
X_train.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean
id,,,,,,,,,,,,,,,,,
186150,284024,284025,"What is the difference between ""visa on arriva...",Do I need a visa to visit Ireland from England?,2.067091,5.683037,5.442245,5.445135,1374.537591,1.615564,153.141828,11.181659,153.141828,11.181659,153.141828,11.181658,21.842760
208652,312703,312704,What are the chances of receiving invitation t...,What are the chances of receiving invitation t...,3.775314,5.546891,5.190107,5.187903,1378.097026,0.000000,293.088385,21.479750,293.088385,21.479750,293.088385,21.479750,115.363998
268163,385651,385652,"Is the phrase ""the pressure in an incompressib...",What is the use of Bash in Windows?,3.612430,5.561530,5.419721,5.423611,1304.791055,1.700451,192.698691,14.652063,192.698691,14.652063,192.698691,14.652064,50.769622
155539,243655,243656,How do I buy goods from Amazon and ship to Vie...,How can I sell Vietnam traditional silk ties o...,3.807038,6.437643,6.175931,6.171602,1521.667788,1.885054,210.650291,15.631263,210.650291,15.631263,210.650291,15.631262,44.743726
212779,317931,317932,How do you read tarot cards?,How do I read tarot cards?,0.900861,1.929870,1.858503,1.860104,483.393402,0.000000,61.085482,4.502494,61.085482,4.502494,61.085482,4.502494,8.293417


In [82]:
X_train[X_train.isnull().any(axis=1)]

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean
id,,,,,,,,,,,,,,,,,
13587,26077,26078,Why did you cry?,"In ten words or less, what makes you cry?",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239614,173260,3007,"In mathematics, how many zeros does a million ...","If 204=8, 503=13, 305=11 and 907=25, what is 705?",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375111,506098,1896,Why is 1/0=∞?,Why does zero factorial (0!) equal one (1)?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43536,78296,61064,What is 9*3?,What is 3+3*3+3?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253902,368527,368528,Yf v. B.?,Name written as x kr y till 12…nw its written ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174113,268421,268422,Hi how are you doing?,Is this good to learn AS400..? how is the futu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360644,490443,490444,Why am I here?,Have you ever asked to yourself why do you exist?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348201,476742,476743,What is the best way to describe yourself?,How do you describe yourself?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178801,274516,1601,What are some good reasons for going back with...,How do I get my ex back?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Test set

In [15]:
#q_meta_test = [(q1_len, q2_len) for q1_len, q2_len in zip(get_q_lengths(X1_ft_test), get_q_lengths(X2_ft_test))]

In [16]:
#pickle.dump(q_meta_test, open(data_folder+'q_meta_test.p', 'wb'))

In [ ]:
#ps.copy_file(dest_bucket=INPUT_BUCKET, file='q_meta_test.p', source='q_meta_test.p')

In [17]:
#X_test_300 = np.concatenate( 
#    np.vstack( [np.array(np.vsplit(y, y.shape[0])) for y in (x for x in X_ft_test if x.size>0)] )
#)

In [18]:
#X_test_300.shape

(1285060, 300)

In [19]:
#from scipy.io import mmwrite, mmread
#mmwrite( 'wor2vec_300_full_test.mtx', X_test_300 )
#np.savez(data_folder+'wor2vec_300_full_test', data=X_test_300)

In [ ]:
#ps.copy_file(dest_bucket=INPUT_BUCKET, file='wor2vec_300_full_test.mtx', source='wor2vec_300_full_test.mtx')

In [23]:
#del X_test_300, q_meta_test, X_ft_test

In [ ]:
#ps.get_file(bucket=INPUT_BUCKET, filename='embed2_full_test.mtx', filepath='embed2_full_test.mtx')

In [ ]:
#from scipy.io import mmread
#X_rd_test = mmread('embed2_full_test.mtx')

In [ ]:
#X_rd_test.shape

In [ ]:
#ps.get_file(bucket=INPUT_BUCKET, filename='q_meta_test.p', filepath='q_meta_test.p')

In [41]:
#q_meta_test = pickle.load(open(data_folder+'q_meta_test.p','rb'))

In [42]:
#mmwrite( data_folder+'wor2vec_300_full_train.mtx', X_train_300 )
#X_test_300 = np.load(data_folder+'wor2vec_300_full_test.npz')['data']

In [43]:
#X_test_300.shape

(1285060, 300)

In [44]:
# rebuild X1_rd_test and X2_rd_test
#X1_list = []
#X2_list = []
#X1_rd_list = []
#X2_rd_list = []
#q1_ptr = 0
#for len_q1, _ in q_meta_test:
#    q1 = np.array(X_test_300[q1_ptr:q1_ptr+len_q1])
#    #q1_rd = np.array(X_rd_test[q1_ptr:q1_ptr+len_q1])
#    X1_list.append(q1)
#    #X1_rd_list.append(q1_rd)
#    q1_ptr = q1_ptr+len_q1
#q2_ptr = q1_ptr
#for _, len_q2 in q_meta_test:
#    q2 = np.array(X_test_300[q2_ptr:q2_ptr+len_q2])
#    #q2_rd = np.array(X_rd_test[q2_ptr:q2_ptr+len_q2])
#    X2_list.append(q2)
#    #X2_rd_list.append(q2_rd)
#    q2_ptr = q2_ptr+len_q2
#X1_test = np.array(X1_list)
#X2_test = np.array(X2_list)
#X1_rd_test = np.array(X1_rd_list)
#X2_rd_test = np.array(X2_rd_list)

In [46]:
#X2_test.shape

(133415,)

In [48]:
#del X1_list, X2_list, q_meta_test, X_test_300, X1_rd_list, X2_rd_list

In [1]:
X1_test_w = pickle.load(open(data_folder+'X1_test_w.p','rb'))
X2_test_w = pickle.load(open(data_folder+'X2_test_w.p','rb'))

NameError: name 'pickle' is not defined

In [1]:
X1_test = pickle.load(open(data_folder+'X1_test.p','rb'))
X2_test = pickle.load(open(data_folder+'X2_test.p','rb'))

NameError: name 'pickle' is not defined

In [ ]:
#jaccard = compute(*jaccard)

In [49]:
#chebyshev = compute(*chebyshev)
#compute_and_save(X1_test, X2_test, 'chebyshev', 'chebyshev_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'chebyshev', 'chebyshev_test_w', data_folder)

In [50]:
#braycurtis = compute(*braycurtis)
#compute_and_save(X1_test, X2_test, 'braycurtis', 'braycurtis_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'braycurtis', 'braycurtis_test_w', data_folder)

distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

In [52]:
#cosine = compute(*cosine)
#compute_and_save(X1_test, X2_test, 'cosine', 'cosine_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'cosine', 'cosine_test_w', data_folder)

distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

In [53]:
#correlation = compute(*correlation)
#compute_and_save(X1_test, X2_test, 'correlation', 'correlation_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'correlation', 'correlation_test_w', data_folder)

distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

In [ ]:
#hamming = compute(*hamming)

In [54]:
#canberra = compute(*canberra)
#compute_and_save(X1_test, X2_test, 'canberra', 'canberra_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'canberra', 'canberra_test_w', data_folder)

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

In [55]:
#hausdorff = compute(*hausdorff)
#compute_and_save(X1_test, X2_test, 'hausdorff', 'hausdorff_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'hausdorff', 'hausdorff_test_w', data_folder)

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-213ebfd3-b762-40d0-9e56-d6de1b2d1059
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-34168ee2-87ae-47fd-af64-6cef9a45da90
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-a76ecf14-7bef-4db6-9834-a6961d94d1e4
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-7b7ac2e1-3618-4e45-be9d-8f624a8c302a
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-9d483136-a90d-44dd-b3ad-85aeaa721eb0
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-10b68423-fa81-4735-b205-95

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-758aeea6-e6d5-4cdb-9c9f-91768282b58a
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-29348394-fb93-4f4f-b27c-cf1daaa41dc4
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-c4ebab75-34c8-4059-a28f-eefb2de50f5c
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-cb81a54f-7ab3-40e1-b92b-e223320bcc7b
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-05361047-af39-41ed-88f3-1c723848ff0a
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-ebd8ce40-b882-488e-be47-52

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-ef85aeeb-66f4-42c4-a029-42280fa2fb15
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-3cb1a200-287b-43c6-8656-c28d54b4df24
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-870fb19a-f8f2-4d5c-81ad-78dbba1fcd4f
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-1b46e106-9c1b-4fb6-8456-24ffeb00e1c5
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-7f2b52f3-8687-44e4-ba46-7871f2a588fc
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-47321146-d9d7-4f92-85cd-8b

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-bb6b93b8-ed06-4269-97ac-e50751362204
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-8b5445de-9abb-4f1a-b047-b5c4c4535ac5
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-c5710a3e-d55a-42f6-82d2-e8999ad5062d
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-57781643-27ef-4f01-b219-54bac7216594
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-b7cfbf02-c734-4192-a57a-d223c700d22c
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-b53e5c0c-6a1e-40b0-a4c1-3b

...

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-c92dae5a-70b1-4039-be63-e00dcbb424fe
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-2dbb2eaa-7382-45eb-bdf0-46aea200a4f0
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-d0bb4182-969c-4272-9c09-45da4fbc5802
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-96adaf80-e941-439e-9106-4ea10aadeb6a
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-ab8f85f3-fb06-488b-b94b-2422aba695f3
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-9e92f188-2387-495f-a49b-d4

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-d859e2fc-a452-4dc6-8688-b8a948c30891
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-fa689104-6d00-42b4-8375-0e4d543ab4f6
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-1e2b20d1-4f1d-4bbc-9fed-09fc239181f5
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-4d266048-c39b-4eac-b7b7-ecac2e538348
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-66bfceb6-96e1-4627-8a5e-fb95057023af
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-600b9017-c550-43f6-b0cb-06

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-177ccaa3-ae5d-46c3-be41-4b1d5f11fe2b
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-eaaf9c3b-602d-4ea0-b321-dcc2523d7df2
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-4250d34a-7023-4ff4-a1e1-44b7caf99c10
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-983217f5-ef3e-4dcc-b95a-cca60e71bb34
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-d96d6e56-5d78-4ad9-9f26-9eb087568948
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-35c9d7ef-72f4-4f37-a6a7-92

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-5fb4824d-ec1a-46bd-990f-359bb3ab6a02
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-aa872406-b1c1-438a-a795-b9340ea858fc
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-7de08217-fa94-4bc9-8912-14005074af09
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-546c3aaf-805f-4344-9918-2211854a4f83
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-113b05a7-4641-4a7e-86a3-bb61b1b73de4
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-7c7a25be-031c-4e90-94d7-ac

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-5c44ab6a-7958-4e56-938f-5f2bc99c659d
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-655baaf8-0456-4985-8cda-d8f78c9b1d0d
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-abb05e00-6b13-4fea-820f-30e3db2c391e
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-46ef6e74-2698-40e7-9333-5a6aab7c9174
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-2fdfcf94-4403-4feb-b040-43cef3b23337
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://192.168.254.70:39217'], compute_pairwise_dist-cce1b683-e983-4fdd-b315-f8

In [56]:
#cityblock = compute(*cityblock)
#compute_and_save(X1_test, X2_test, 'cityblock', 'cityblock_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'cityblock', 'cityblock_test_w', data_folder)

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


In [57]:
#euclidean = compute(*euclidean)
#compute_and_save(X1_test, X2_test, 'euclidean', 'euclidean_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'euclidean', 'euclidean_test_w', data_folder)

In [58]:
#l1 = compute(*l1)
#compute_and_save(X1_test, X2_test, 'l1', 'l1_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'l1', 'l1_test_w', data_folder)

In [59]:
#l2 = compute(*l2)
#compute_and_save(X1_test, X2_test, 'l2', 'l2_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'l2', 'l2_test_w', data_folder)

In [60]:
#manhattan = compute(*manhattan)
#compute_and_save(X1_test, X2_test, 'manhattan', 'manhattan_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'manhattan', 'manhattan_test_w', data_folder)

In [ ]:
#dice = compute(*dice)

In [ ]:
#kulsinski = compute(*kulsinski)

In [ ]:
#rogerstanimoto = compute(*rogerstanimoto)

In [ ]:
#russellrao = compute(*russellrao)

In [ ]:
#sokalmichener = compute(*sokalmichener)

In [61]:
#minkowski = compute(*minkowski)
#compute_and_save(X1_test, X2_test, 'minkowski', 'minkowski_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'minkowski', 'minkowski_test_w', data_folder)

In [ ]:
#seuclidean = compute(*seuclidean)

In [ ]:
#sokalsneath = compute(*sokalsneath)

In [62]:
#sqeuclidean = compute(*sqeuclidean)
#compute_and_save(X1_test, X2_test, 'sqeuclidean', 'sqeuclidean_test', data_folder)
compute_and_save(X1_test, X2_test, X1_test_w, X2_test_w, 'sqeuclidean', 'sqeuclidean_test_w', data_folder)

In [ ]:
#fdtw = compute(*fdtw)

In [ ]:
#dtw = compute(*dtw)

In [ ]:
#pcm = compute(*pcm)

In [ ]:
#area = compute(*area)

In [ ]:
#curve_length = compute(*curve_length)

In [ ]:
#discrete_frechet = compute(*discrete_frechet)

In [64]:
del X1_test, X2_test, X1_test_w, X2_test_w

In [83]:
X_test = add_column(X_test, 'chebyshev', 'test')

In [84]:
X_test = add_column(X_test, 'braycurtis', 'test')

In [85]:
X_test = add_column(X_test, 'cosine', 'test')

In [86]:
X_test = add_column(X_test, 'correlation', 'test')

In [87]:
X_test = add_column(X_test, 'canberra', 'test')

In [88]:
X_test = add_column(X_test, 'hausdorff', 'test')

In [89]:
X_test = add_column(X_test, 'cityblock', 'test')

In [90]:
X_test = add_column(X_test, 'euclidean', 'test')

In [91]:
X_test = add_column(X_test, 'l1', 'test')

In [92]:
X_test = add_column(X_test, 'l2', 'test')

In [93]:
X_test = add_column(X_test, 'manhattan', 'test')

In [94]:
X_test = add_column(X_test, 'minkowski', 'test')

In [95]:
X_test = add_column(X_test, 'sqeuclidean', 'test')

In [96]:
X_test.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean
id,,,,,,,,,,,,,,,,,
8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,1.043210,2.279212,2.258787,2.251032,546.958943,1.346215,63.292508,4.725193,63.292508,4.725193,63.292508,4.725193,8.600123
224279,332326,332327,Will a breathing treatment help a cough?,How can I help someone that is unconscious but...,0.744009,2.812987,2.728349,2.724919,670.072040,1.331043,54.392818,4.009771,54.392818,4.009771,54.392818,4.009771,5.428841
252452,336023,366789,Is Kellyanne Conway annoying in your opinion?,Did Kellyanne Conway really imply that we shou...,1.732130,5.815354,5.744035,5.741251,1341.080766,0.935798,118.726409,8.696970,118.726409,8.696970,118.726409,8.696970,15.124150
174039,268330,268331,How do you rate (1-10) and review Maruti Baleno?,What career options does one have after comple...,2.012377,6.013366,5.812397,5.809918,1408.605420,1.479857,141.482726,10.386573,141.482726,10.386573,141.482726,10.386573,17.130735
384863,28901,233483,What are some good books on marketing?,What are some of the best books ever written a...,0.783164,2.678354,2.513877,2.514548,651.312923,1.055390,56.638747,4.125052,56.638747,4.125052,56.638747,4.125052,5.567281


In [97]:
X_test[X_test.isnull().any(axis=1)]

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean
id,,,,,,,,,,,,,,,,,
394708,47433,527619,When and how will atrocities on Hindu dalits s...,If (x+y) =7 and xy=127 find x^2-y^2?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393288,3007,193368,"If 204=8, 503=13, 305=11 and 907=25, what is 705?",Can you add 5 odd numbers to get 30?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110983,68218,181872,Why do we say hi?,"Why do we say ""hi"" when we talk with others?",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318371,443715,35958,What is)'(?,"What is ""what is""?",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321352,326142,165846,What is 4 2/3-1 1/3÷2?,What is 1/ (3^1/2)?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166112,257824,257825,"What are B2B, B2C, C2B and C2C?","If B2B has ""business customers"", then what doe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316274,99469,441288,"I have no interest in anything, what should I do?","Don't have interest in anything, What to do?",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295593,58523,326142,What is 2^ (1/2!) ^ (1/3!) ^ (1/4!)^...?,What is 4 2/3-1 1/3÷2?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71249,122645,122646,How should I get up at 4 am daily?,How can I get up at 4 am?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Fuzzy-wuzzy

In [98]:
# difference in text size
compute_size_diff = lambda row: abs(len(str(row['question1'])) - len(str(row['question2'])))
X_train['size_diff'] = X_train.apply(compute_size_diff, axis=1)
X_train.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean,size_diff
id,,,,,,,,,,,,,,,,,,
186150,284024,284025,"What is the difference between ""visa on arriva...",Do I need a visa to visit Ireland from England?,2.067091,5.683037,5.442245,5.445135,1374.537591,1.615564,153.141828,11.181659,153.141828,11.181659,153.141828,11.181658,21.842760,56
208652,312703,312704,What are the chances of receiving invitation t...,What are the chances of receiving invitation t...,3.775314,5.546891,5.190107,5.187903,1378.097026,0.000000,293.088385,21.479750,293.088385,21.479750,293.088385,21.479750,115.363998,0
268163,385651,385652,"Is the phrase ""the pressure in an incompressib...",What is the use of Bash in Windows?,3.612430,5.561530,5.419721,5.423611,1304.791055,1.700451,192.698691,14.652063,192.698691,14.652063,192.698691,14.652064,50.769622,115
155539,243655,243656,How do I buy goods from Amazon and ship to Vie...,How can I sell Vietnam traditional silk ties o...,3.807038,6.437643,6.175931,6.171602,1521.667788,1.885054,210.650291,15.631263,210.650291,15.631263,210.650291,15.631262,44.743726,58
212779,317931,317932,How do you read tarot cards?,How do I read tarot cards?,0.900861,1.929870,1.858503,1.860104,483.393402,0.000000,61.085482,4.502494,61.085482,4.502494,61.085482,4.502494,8.293417,2


In [99]:
from fuzzywuzzy import fuzz

In [100]:
# ratio
compute_ratio = lambda row: fuzz.ratio(str(row['question1']), str(row['question2']))
X_train['ratio'] = X_train.apply(compute_ratio, axis=1)
X_train.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean,size_diff,ratio
id,,,,,,,,,,,,,,,,,,,
186150,284024,284025,"What is the difference between ""visa on arriva...",Do I need a visa to visit Ireland from England?,2.067091,5.683037,5.442245,5.445135,1374.537591,1.615564,153.141828,11.181659,153.141828,11.181659,153.141828,11.181658,21.842760,56,37
208652,312703,312704,What are the chances of receiving invitation t...,What are the chances of receiving invitation t...,3.775314,5.546891,5.190107,5.187903,1378.097026,0.000000,293.088385,21.479750,293.088385,21.479750,293.088385,21.479750,115.363998,0,99
268163,385651,385652,"Is the phrase ""the pressure in an incompressib...",What is the use of Bash in Windows?,3.612430,5.561530,5.419721,5.423611,1304.791055,1.700451,192.698691,14.652063,192.698691,14.652063,192.698691,14.652064,50.769622,115,25
155539,243655,243656,How do I buy goods from Amazon and ship to Vie...,How can I sell Vietnam traditional silk ties o...,3.807038,6.437643,6.175931,6.171602,1521.667788,1.885054,210.650291,15.631263,210.650291,15.631263,210.650291,15.631262,44.743726,58,48
212779,317931,317932,How do you read tarot cards?,How do I read tarot cards?,0.900861,1.929870,1.858503,1.860104,483.393402,0.000000,61.085482,4.502494,61.085482,4.502494,61.085482,4.502494,8.293417,2,93


In [101]:
# partial ratio
compute_partial_ratio = lambda row: fuzz.partial_ratio(str(row['question1']), str(row['question2']))
X_train['partial_ratio'] = X_train.apply(compute_partial_ratio, axis=1)
X_train.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean,size_diff,ratio,partial_ratio
id,,,,,,,,,,,,,,,,,,,,
186150,284024,284025,"What is the difference between ""visa on arriva...",Do I need a visa to visit Ireland from England?,2.067091,5.683037,5.442245,5.445135,1374.537591,1.615564,153.141828,11.181659,153.141828,11.181659,153.141828,11.181658,21.842760,56,37,47
208652,312703,312704,What are the chances of receiving invitation t...,What are the chances of receiving invitation t...,3.775314,5.546891,5.190107,5.187903,1378.097026,0.000000,293.088385,21.479750,293.088385,21.479750,293.088385,21.479750,115.363998,0,99,99
268163,385651,385652,"Is the phrase ""the pressure in an incompressib...",What is the use of Bash in Windows?,3.612430,5.561530,5.419721,5.423611,1304.791055,1.700451,192.698691,14.652063,192.698691,14.652063,192.698691,14.652064,50.769622,115,25,51
155539,243655,243656,How do I buy goods from Amazon and ship to Vie...,How can I sell Vietnam traditional silk ties o...,3.807038,6.437643,6.175931,6.171602,1521.667788,1.885054,210.650291,15.631263,210.650291,15.631263,210.650291,15.631262,44.743726,58,48,51
212779,317931,317932,How do you read tarot cards?,How do I read tarot cards?,0.900861,1.929870,1.858503,1.860104,483.393402,0.000000,61.085482,4.502494,61.085482,4.502494,61.085482,4.502494,8.293417,2,93,88


In [102]:
# token_sort_ratio
compute_token_sort_ratio = lambda row: fuzz.token_sort_ratio(str(row['question1']), str(row['question2']))
X_train['token_sort_ratio'] = X_train.apply(compute_token_sort_ratio, axis=1)
X_train.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,...,euclidean,l1,l2,manhattan,minkowski,sqeuclidean,size_diff,ratio,partial_ratio,token_sort_ratio
id,,,,,,,,,,,,,,,,,,,,,
186150,284024,284025,"What is the difference between ""visa on arriva...",Do I need a visa to visit Ireland from England?,2.067091,5.683037,5.442245,5.445135,1374.537591,1.615564,...,11.181659,153.141828,11.181659,153.141828,11.181658,21.842760,56,37,47,44
208652,312703,312704,What are the chances of receiving invitation t...,What are the chances of receiving invitation t...,3.775314,5.546891,5.190107,5.187903,1378.097026,0.000000,...,21.479750,293.088385,21.479750,293.088385,21.479750,115.363998,0,99,99,99
268163,385651,385652,"Is the phrase ""the pressure in an incompressib...",What is the use of Bash in Windows?,3.612430,5.561530,5.419721,5.423611,1304.791055,1.700451,...,14.652063,192.698691,14.652063,192.698691,14.652064,50.769622,115,25,51,25
155539,243655,243656,How do I buy goods from Amazon and ship to Vie...,How can I sell Vietnam traditional silk ties o...,3.807038,6.437643,6.175931,6.171602,1521.667788,1.885054,...,15.631263,210.650291,15.631263,210.650291,15.631262,44.743726,58,48,51,54
212779,317931,317932,How do you read tarot cards?,How do I read tarot cards?,0.900861,1.929870,1.858503,1.860104,483.393402,0.000000,...,4.502494,61.085482,4.502494,61.085482,4.502494,8.293417,2,93,88,88


In [103]:
# token_set_ratio
compute_token_set_ratio = lambda row: fuzz.token_set_ratio(str(row['question1']), str(row['question2']))
X_train['token_set_ratio'] = X_train.apply(compute_token_set_ratio, axis=1)
X_train.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,...,l1,l2,manhattan,minkowski,sqeuclidean,size_diff,ratio,partial_ratio,token_sort_ratio,token_set_ratio
id,,,,,,,,,,,,,,,,,,,,,
186150,284024,284025,"What is the difference between ""visa on arriva...",Do I need a visa to visit Ireland from England?,2.067091,5.683037,5.442245,5.445135,1374.537591,1.615564,...,153.141828,11.181659,153.141828,11.181658,21.842760,56,37,47,44,43
208652,312703,312704,What are the chances of receiving invitation t...,What are the chances of receiving invitation t...,3.775314,5.546891,5.190107,5.187903,1378.097026,0.000000,...,293.088385,21.479750,293.088385,21.479750,115.363998,0,99,99,99,99
268163,385651,385652,"Is the phrase ""the pressure in an incompressib...",What is the use of Bash in Windows?,3.612430,5.561530,5.419721,5.423611,1304.791055,1.700451,...,192.698691,14.652063,192.698691,14.652064,50.769622,115,25,51,25,42
155539,243655,243656,How do I buy goods from Amazon and ship to Vie...,How can I sell Vietnam traditional silk ties o...,3.807038,6.437643,6.175931,6.171602,1521.667788,1.885054,...,210.650291,15.631263,210.650291,15.631262,44.743726,58,48,51,54,67
212779,317931,317932,How do you read tarot cards?,How do I read tarot cards?,0.900861,1.929870,1.858503,1.860104,483.393402,0.000000,...,61.085482,4.502494,61.085482,4.502494,8.293417,2,93,88,88,96


In [104]:
# build complete feature dataframe
#X_train_temp = pd.concat([pd.DataFrame(X1, columns=['q1_'+str(i) for i in range(X1.shape[1])], index=X_train.index), 
#                     pd.DataFrame(X2, columns=['q2_'+str(i) for i in range(X2.shape[1])], index=X_train.index)], axis=1)
#X_train_temp.head()

In [105]:
#X_train = pd.concat([X_train_temp, X_train], axis=1)
#del X_train_temp
X_train_final = X_train.drop(columns=['qid1', 'qid2','question1','question2']).dropna()
X_train_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270404 entries, 186150 to 121959
Data columns (total 18 columns):
chebyshev           270404 non-null float64
braycurtis          270404 non-null float64
cosine              270404 non-null float64
correlation         270404 non-null float64
canberra            270404 non-null float64
hausdorff           270404 non-null float64
cityblock           270404 non-null float64
euclidean           270404 non-null float64
l1                  270404 non-null float64
l2                  270404 non-null float64
manhattan           270404 non-null float64
minkowski           270404 non-null float64
sqeuclidean         270404 non-null float64
size_diff           270404 non-null int64
ratio               270404 non-null int64
partial_ratio       270404 non-null int64
token_sort_ratio    270404 non-null int64
token_set_ratio     270404 non-null int64
dtypes: float64(13), int64(5)
memory usage: 39.2 MB


In [106]:
X_train_final.tail(20)

,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean,size_diff,ratio,partial_ratio,token_sort_ratio,token_set_ratio
id,,,,,,,,,,,,,,,,,,
262915,2.395697,5.645886,5.437570,5.432416,1324.947809,2.789291,177.971496,12.962920,177.971496,12.962920,177.971496,12.962919,30.975481,31,42,43,41,46
64820,2.088093,5.429932,5.160242,5.156631,1283.576483,1.366273,158.525298,11.486521,158.525298,11.486521,158.525298,11.486520,26.562313,8,69,70,70,79
329367,3.071903,8.784382,8.539742,8.538970,2031.214761,2.865792,213.163837,15.584897,213.163837,15.584897,213.163837,15.584897,32.590289,190,22,49,25,43
41090,0.926751,3.083536,2.917314,2.903381,757.927499,0.885681,71.485013,5.212074,71.485013,5.212074,71.485013,5.212074,7.926915,18,75,84,67,91
278169,1.090599,2.569957,2.446168,2.447597,606.049494,1.233788,74.130409,5.449799,74.130409,5.449799,74.130409,5.449799,13.454010,22,45,53,60,65
191336,1.502752,2.767397,2.709365,2.709006,661.474254,1.345086,111.303728,8.206067,111.303728,8.206067,111.303728,8.206067,22.546543,12,47,51,52,56
175204,2.460463,4.576073,4.326492,4.325745,1106.681375,1.000259,134.024228,10.106836,134.024228,10.106836,134.024228,10.106836,32.994585,9,78,82,74,88
388471,3.184504,7.267180,7.017347,7.016691,1694.606045,1.800405,206.451623,15.350261,206.451623,15.350261,206.451623,15.350261,48.218635,20,43,43,52,56
374874,3.197301,7.843037,7.673778,7.673166,1834.149975,2.864478,218.848712,16.109211,218.848712,16.109211,218.848712,16.109210,38.104299,29,37,39,43,43


##### Test set

In [107]:
# difference in text size
X_test['size_diff'] = X_test.apply(compute_size_diff, axis=1)
X_test.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean,size_diff
id,,,,,,,,,,,,,,,,,,
8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,1.043210,2.279212,2.258787,2.251032,546.958943,1.346215,63.292508,4.725193,63.292508,4.725193,63.292508,4.725193,8.600123,0
224279,332326,332327,Will a breathing treatment help a cough?,How can I help someone that is unconscious but...,0.744009,2.812987,2.728349,2.724919,670.072040,1.331043,54.392818,4.009771,54.392818,4.009771,54.392818,4.009771,5.428841,23
252452,336023,366789,Is Kellyanne Conway annoying in your opinion?,Did Kellyanne Conway really imply that we shou...,1.732130,5.815354,5.744035,5.741251,1341.080766,0.935798,118.726409,8.696970,118.726409,8.696970,118.726409,8.696970,15.124150,73
174039,268330,268331,How do you rate (1-10) and review Maruti Baleno?,What career options does one have after comple...,2.012377,6.013366,5.812397,5.809918,1408.605420,1.479857,141.482726,10.386573,141.482726,10.386573,141.482726,10.386573,17.130735,79
384863,28901,233483,What are some good books on marketing?,What are some of the best books ever written a...,0.783164,2.678354,2.513877,2.514548,651.312923,1.055390,56.638747,4.125052,56.638747,4.125052,56.638747,4.125052,5.567281,23


In [108]:
# ratio
X_test['ratio'] = X_test.apply(compute_ratio, axis=1)
X_test.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean,size_diff,ratio
id,,,,,,,,,,,,,,,,,,,
8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,1.043210,2.279212,2.258787,2.251032,546.958943,1.346215,63.292508,4.725193,63.292508,4.725193,63.292508,4.725193,8.600123,0,88
224279,332326,332327,Will a breathing treatment help a cough?,How can I help someone that is unconscious but...,0.744009,2.812987,2.728349,2.724919,670.072040,1.331043,54.392818,4.009771,54.392818,4.009771,54.392818,4.009771,5.428841,23,33
252452,336023,366789,Is Kellyanne Conway annoying in your opinion?,Did Kellyanne Conway really imply that we shou...,1.732130,5.815354,5.744035,5.741251,1341.080766,0.935798,118.726409,8.696970,118.726409,8.696970,118.726409,8.696970,15.124150,73,40
174039,268330,268331,How do you rate (1-10) and review Maruti Baleno?,What career options does one have after comple...,2.012377,6.013366,5.812397,5.809918,1408.605420,1.479857,141.482726,10.386573,141.482726,10.386573,141.482726,10.386573,17.130735,79,30
384863,28901,233483,What are some good books on marketing?,What are some of the best books ever written a...,0.783164,2.678354,2.513877,2.514548,651.312923,1.055390,56.638747,4.125052,56.638747,4.125052,56.638747,4.125052,5.567281,23,69


In [109]:
# partial ratio
X_test['partial_ratio'] = X_test.apply(compute_partial_ratio, axis=1)
X_test.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,cityblock,euclidean,l1,l2,manhattan,minkowski,sqeuclidean,size_diff,ratio,partial_ratio
id,,,,,,,,,,,,,,,,,,,,
8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,1.043210,2.279212,2.258787,2.251032,546.958943,1.346215,63.292508,4.725193,63.292508,4.725193,63.292508,4.725193,8.600123,0,88,88
224279,332326,332327,Will a breathing treatment help a cough?,How can I help someone that is unconscious but...,0.744009,2.812987,2.728349,2.724919,670.072040,1.331043,54.392818,4.009771,54.392818,4.009771,54.392818,4.009771,5.428841,23,33,38
252452,336023,366789,Is Kellyanne Conway annoying in your opinion?,Did Kellyanne Conway really imply that we shou...,1.732130,5.815354,5.744035,5.741251,1341.080766,0.935798,118.726409,8.696970,118.726409,8.696970,118.726409,8.696970,15.124150,73,40,56
174039,268330,268331,How do you rate (1-10) and review Maruti Baleno?,What career options does one have after comple...,2.012377,6.013366,5.812397,5.809918,1408.605420,1.479857,141.482726,10.386573,141.482726,10.386573,141.482726,10.386573,17.130735,79,30,42
384863,28901,233483,What are some good books on marketing?,What are some of the best books ever written a...,0.783164,2.678354,2.513877,2.514548,651.312923,1.055390,56.638747,4.125052,56.638747,4.125052,56.638747,4.125052,5.567281,23,69,61


In [110]:
# token_sort_ratio
X_test['token_sort_ratio'] = X_test.apply(compute_token_sort_ratio, axis=1)
X_test.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,...,euclidean,l1,l2,manhattan,minkowski,sqeuclidean,size_diff,ratio,partial_ratio,token_sort_ratio
id,,,,,,,,,,,,,,,,,,,,,
8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,1.043210,2.279212,2.258787,2.251032,546.958943,1.346215,...,4.725193,63.292508,4.725193,63.292508,4.725193,8.600123,0,88,88,81
224279,332326,332327,Will a breathing treatment help a cough?,How can I help someone that is unconscious but...,0.744009,2.812987,2.728349,2.724919,670.072040,1.331043,...,4.009771,54.392818,4.009771,54.392818,4.009771,5.428841,23,33,38,48
252452,336023,366789,Is Kellyanne Conway annoying in your opinion?,Did Kellyanne Conway really imply that we shou...,1.732130,5.815354,5.744035,5.741251,1341.080766,0.935798,...,8.696970,118.726409,8.696970,118.726409,8.696970,15.124150,73,40,56,43
174039,268330,268331,How do you rate (1-10) and review Maruti Baleno?,What career options does one have after comple...,2.012377,6.013366,5.812397,5.809918,1408.605420,1.479857,...,10.386573,141.482726,10.386573,141.482726,10.386573,17.130735,79,30,42,32
384863,28901,233483,What are some good books on marketing?,What are some of the best books ever written a...,0.783164,2.678354,2.513877,2.514548,651.312923,1.055390,...,4.125052,56.638747,4.125052,56.638747,4.125052,5.567281,23,69,61,66


In [111]:
# token_set_ratio
X_test['token_set_ratio'] = X_test.apply(compute_token_set_ratio, axis=1)
X_test.head()

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,...,l1,l2,manhattan,minkowski,sqeuclidean,size_diff,ratio,partial_ratio,token_sort_ratio,token_set_ratio
id,,,,,,,,,,,,,,,,,,,,,
8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,1.043210,2.279212,2.258787,2.251032,546.958943,1.346215,...,63.292508,4.725193,63.292508,4.725193,8.600123,0,88,88,81,90
224279,332326,332327,Will a breathing treatment help a cough?,How can I help someone that is unconscious but...,0.744009,2.812987,2.728349,2.724919,670.072040,1.331043,...,54.392818,4.009771,54.392818,4.009771,5.428841,23,33,38,48,55
252452,336023,366789,Is Kellyanne Conway annoying in your opinion?,Did Kellyanne Conway really imply that we shou...,1.732130,5.815354,5.744035,5.741251,1341.080766,0.935798,...,118.726409,8.696970,118.726409,8.696970,15.124150,73,40,56,43,53
174039,268330,268331,How do you rate (1-10) and review Maruti Baleno?,What career options does one have after comple...,2.012377,6.013366,5.812397,5.809918,1408.605420,1.479857,...,141.482726,10.386573,141.482726,10.386573,17.130735,79,30,42,32,33
384863,28901,233483,What are some good books on marketing?,What are some of the best books ever written a...,0.783164,2.678354,2.513877,2.514548,651.312923,1.055390,...,56.638747,4.125052,56.638747,4.125052,5.567281,23,69,61,66,88


In [112]:
X_test_final = X_test.drop(columns=['question1','question2', 'qid1', 'qid2']).dropna()
X_test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133178 entries, 8067 to 346580
Data columns (total 18 columns):
chebyshev           133178 non-null float64
braycurtis          133178 non-null float64
cosine              133178 non-null float64
correlation         133178 non-null float64
canberra            133178 non-null float64
hausdorff           133178 non-null float64
cityblock           133178 non-null float64
euclidean           133178 non-null float64
l1                  133178 non-null float64
l2                  133178 non-null float64
manhattan           133178 non-null float64
minkowski           133178 non-null float64
sqeuclidean         133178 non-null float64
size_diff           133178 non-null int64
ratio               133178 non-null int64
partial_ratio       133178 non-null int64
token_sort_ratio    133178 non-null int64
token_set_ratio     133178 non-null int64
dtypes: float64(13), int64(5)
memory usage: 19.3 MB


In [125]:
X_temp = X_train
X_temp['dup'] = y_train
X_temp.head(30)

,qid1,qid2,question1,question2,chebyshev,braycurtis,cosine,correlation,canberra,hausdorff,...,l2,manhattan,minkowski,sqeuclidean,size_diff,ratio,partial_ratio,token_sort_ratio,token_set_ratio,dup
id,,,,,,,,,,,,,,,,,,,,,
186150,284024,284025,"What is the difference between ""visa on arriva...",Do I need a visa to visit Ireland from England?,2.067091,5.683037,5.442245,5.445135,1374.537591,1.615564,...,11.181659,153.141828,11.181658,21.842760,56,37,47,44,43,0
208652,312703,312704,What are the chances of receiving invitation t...,What are the chances of receiving invitation t...,3.775314,5.546891,5.190107,5.187903,1378.097026,0.000000,...,21.479750,293.088385,21.479750,115.363998,0,99,99,99,99,0
268163,385651,385652,"Is the phrase ""the pressure in an incompressib...",What is the use of Bash in Windows?,3.612430,5.561530,5.419721,5.423611,1304.791055,1.700451,...,14.652063,192.698691,14.652064,50.769622,115,25,51,25,42,0
155539,243655,243656,How do I buy goods from Amazon and ship to Vie...,How can I sell Vietnam traditional silk ties o...,3.807038,6.437643,6.175931,6.171602,1521.667788,1.885054,...,15.631263,210.650291,15.631262,44.743726,58,48,51,54,67,0
212779,317931,317932,How do you read tarot cards?,How do I read tarot cards?,0.900861,1.929870,1.858503,1.860104,483.393402,0.000000,...,4.502494,61.085482,4.502494,8.293417,2,93,88,88,96,0
283194,403246,403247,What career path should I take if I'm planning...,What should I take in 11th std as I am plannin...,1.659356,3.060869,2.941370,2.937405,711.465255,2.204727,...,8.402883,115.594536,8.402883,23.289126,2,64,63,63,68,0
223767,105045,331665,Is blood pressure 160/100 very high?,What precautions should I take during high blo...,0.751444,2.431634,2.308958,2.309027,610.076762,0.000000,...,4.120188,55.789641,4.120188,5.840152,22,36,47,52,70,0
349643,116087,19699,How can I see who my boyfriend views on instag...,Can people see if you have viewed their instag...,0.840119,2.415744,2.323275,2.321711,574.910079,0.735438,...,3.735022,50.287909,3.735021,5.674523,0,58,59,61,61,1
273491,391958,391959,What are good Speakers for projector of classr...,Which is the best gym in vadapalani (Chennai) ...,2.580830,5.879139,5.723269,5.719573,1375.655303,1.927784,...,12.464850,170.165686,12.464850,28.749343,41,32,35,36,38,0


# Modeling

### Logistic Regression

In [113]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
logr_model = LogisticRegression(random_state=42)
param_grid = {'C': np.logspace(-2, 7, 10),
             #'penalty': ['l1','l2'],
             'tol': np.logspace(-5, -1, 5),
             #'solver': ['lbfgs']
             #'max_iter': np.linspace(10, 1000, 10)
             }
logr_cv = RandomizedSearchCV(logr_model, param_distributions=param_grid, cv=5, n_jobs=-1)
y_train_final = y_train.loc[X_train_final.index]
logr_cv.fit(X_train_final, y_train_final)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07]), 'tol': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [114]:
logr_cv.best_params_

{'tol': 0.001, 'C': 100.0}

In [115]:
logr_model = LogisticRegression(#solver=logr_cv.best_params_['solver'], 
                                random_state=42, 
                                C=logr_cv.best_params_['C'], 
                                tol=logr_cv.best_params_['tol'], 
                                #max_iter=logr_cv.best_params_['max_iter'], 
                                n_jobs=-1)
logr_model.fit(X_train_final, y_train_final)

LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=42, solver='warn', tol=0.001,
          verbose=0, warm_start=False)

In [116]:
logr_pred = logr_model.predict(X_test_final)
y_test_final = y_test.loc[X_test_final.index]

In [117]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
logr_acc_score = accuracy_score(y_test_final, logr_pred)
logr_prec_score = precision_score(y_test_final, logr_pred)
logr_rec_score = recall_score(y_test_final, logr_pred)
print('Logistic Regression')
print('accuracy score : {}'.format(logr_acc_score))
print('precision score : {}'.format(logr_prec_score))
print('recall score : {}'.format(logr_rec_score))

Logistic Regression
accuracy score : 0.6688492093288682
precision score : 0.5622774028044133
recall score : 0.4597682422662566


### XGBoost

In [118]:
import xgboost as xgb
# Model selection
params_xgb = {'n_estimators' : [1, 2, 4, 8, 16, 32, 64, 100, 200],
               'gamma':np.linspace(.01, 1, 10, endpoint=True), 
               'learning_rate' : np.linspace(.01, 1, 10, endpoint=True),
               'reg_lambda': np.linspace(0.01, 10, 20, endpoint=True),
               'max_depth' : np.linspace(1, 32, 32, endpoint=True, dtype=int)
                 }
cv_xgb = RandomizedSearchCV(xgb.XGBClassifier(objective='binary:logistic', random_state=42), param_distributions=params_xgb, cv=5, n_jobs=3, random_state=42)
cv_xgb.fit(X_train_final, y_train_final)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=10, n_jobs=3,
          param_distributions={'n_estimators': [1, 2, 4, 8, 16, 32, 64, 100, 200], 'gamma': array([0.01, 0.12, 0.23, 0.34, 0.45, 0.56, 0.67, 0.78, 0.89, 1.  ]), 'learning_rate': array([0.01, 0.12, 0.23, 0.34, 0.45, 0.56, 0.67, 0.78, 0.89, 1.  ]), 'reg_lambda': array([ 0.01   ,  0.53579,  1.06158,  1.58737,  2...10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32])},
          pre_dispatch='2*n_jobs', random_state=42, refit

In [119]:
cv_xgb.best_params_

{'reg_lambda': 6.319473684210527,
 'n_estimators': 200,
 'max_depth': 29,
 'learning_rate': 0.23,
 'gamma': 0.23}

In [120]:
clf_xgb_model = xgb.XGBClassifier(random_state=42,
                                  n_estimators=cv_xgb.best_params_['n_estimators'],
                                  gamma=cv_xgb.best_params_['gamma'],
                                  learning_rate=cv_xgb.best_params_['learning_rate'],
                                  reg_lambda=cv_xgb.best_params_['reg_lambda'],
                                  max_depth=cv_xgb.best_params_['max_depth'])
clf_xgb_model.fit(X_train_final, y_train_final)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.23, learning_rate=0.23,
       max_delta_step=0, max_depth=29, min_child_weight=1, missing=None,
       n_estimators=200, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=42, reg_alpha=0,
       reg_lambda=6.319473684210527, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [121]:
y_pred_xgb = clf_xgb_model.predict(X_test_final)
score_xgb = accuracy_score(y_test_final, y_pred_xgb)
rscore_xgb = recall_score(y_test_final, y_pred_xgb)
pscore_xgb = precision_score(y_test_final, y_pred_xgb)
print('Accuracy score for XGBoost ', score_xgb)
print('Recall score for XGBoost ', rscore_xgb)
print('Precision score for XGBoost ', pscore_xgb)

Accuracy score for XGBoost  0.7490351259216987
Recall score for XGBoost  0.6464167158829399
Precision score for XGBoost  0.6640098740638467
